---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    

    import pandas as pd

    import re

    with open('university_towns.txt', 'r') as in_file:
        with open('u_towns.txt', 'w') as out_file:
            for line in in_file:
                line = re.sub('\[[A-Za-z1-9]*\]', '', line)
                line = re.sub('\(.*\)', '', line)
                line = re.sub('.*:', '', line)
                line = re.sub('\(.*', '', line)
                line = re.sub('\[[0-9]*\]', '', line)
                line =re.sub('University Park, Los Angeles', 'Los Angeles', line)
                line = re.sub('Pomona  and formerly Pomona College', 'Pomona', line)
                line = re.sub('University District, San Bernardino', 'San Bernardino', line)
                line = re.sub('Pueblo, Colorado', 'Pueblo', line)
                line = re.sub('Cedar Rapids, Iowa', 'Cedar Rapids', line)
                line = re.sub('Mount Vernon,', 'Mount Vernon', line)
                change = 'Lewiston, Maine' 
                line = re.sub(change, 'Lewiston', line)
                change = 'Faribault, South Central College'
                line = re.sub(change, 'Faribault', line)
                change = 'Mankato , Bethany Lutheran College'
                line = re.sub(change, 'Mankato', line)
                change = 'North Mankato, South Central College'
                line = re.sub(change, 'North Mankato', line)
                change = 'New London, New Hampshire'
                line = re.sub(change, 'New London', line)
                change = 'Morningside Heights, Manhattan'
                line = re.sub(change, 'Manhattan', line)
                change = 'Wilmington, North Carolina'
                line = re.sub(change, 'Wilmington', line)
                change = 'Cecil B. Moore, Philadelphia, also known as \"Templetown\"'
                line = re.sub(change, 'Philadelphia', line)
                change = 'University City, Philadelphia'
                line = re.sub(change, 'Philadelphia', line)
                change = 'Oakland, Pittsburgh'
                line = re.sub(change, 'Pittsburgh', line)
                change = 'University District, Seattle'
                line = re.sub(change, 'Seattle', line)
                out_file.write(line)


    df = pd.read_csv('u_towns.txt', sep=' ', header=None, names=['Cities'])

    def update_state(name):
        states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
        if name in states.values():
            return name
        else:
            return "city"


    df['state'] = df['Cities'].apply(lambda x: update_state(x))

    state_reg = []
    
    def add_state(value):
        states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
        states_in_list = list(states.values())
        if value in states_in_list:
            state_reg.append(value)
            return value
        elif value == 'city':
            return state_reg[-1]

    df['state2'] = df['state'].apply(lambda x: add_state(x))

    df_2 = df.copy()

    df_3 = df_2.drop(df[df['state']==df['state2']].index)

    df_4 = df_3.drop('state', axis=1)

    df_5 = df_4[['state2', 'Cities']]

    df_6 = df_5.rename(columns={'state2': 'State', 'Cities': 'RegionName'})
    
    return df_6

In [6]:
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})


    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    return df_2000['Year, Quarter'].loc[246]

In [8]:
get_recession_start()

/Users/aschharwood/anaconda/envs/ipykernel_py2/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2008q3'

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})

    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    return df_2000['Year, Quarter'].loc[251]

In [10]:
get_recession_end()

/Users/aschharwood/anaconda/envs/ipykernel_py2/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2009q4'

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})

    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    recession_bottom = min(df_2000.loc[246:251]['GDP'])


    return df_2000.loc[df_2000['GDP'] == recession_bottom]['Year, Quarter'].tolist()[0]

In [12]:
get_recession_bottom()

/Users/aschharwood/anaconda/envs/ipykernel_py2/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2009q2'

In [195]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

    import pandas as pd

    df = pd.read_csv('City_Zhvi_AllHomes.csv')


    regions = list(df.columns[0:6])
    years = list(df.columns[51:])
    column_names = regions + years

    df_2 = df[column_names]

    df_3 = df_2.set_index(['RegionID', 'RegionName', 'State', 'Metro','CountyName', 'SizeRank'])

    df_4 = df_3.groupby(pd.to_datetime(df_3.columns).to_period("Q"), axis=1).mean().reset_index()

    df_5 = df_4.drop(['RegionID', 'Metro', 'CountyName','SizeRank'], axis=1)
    
    df_6 = df_5.set_index(['State', 'RegionName'])
    
    df_6.columns =df_6.columns.map(lambda t: str(t).lower())

    return df_6

In [196]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [ ]:
import pandas as pd

df = pd.read_csv('City_Zhvi_AllHomes.csv')


regions = list(df.columns[0:6])
years = list(df.columns[51:])
column_names = regions + years

df_2 = df[column_names]

df_3 = df_2.set_index(['RegionID', 'RegionName', 'State', 'Metro','CountyName', 'SizeRank'])

df_4 = df_3.groupby(pd.to_datetime(df_3.columns).to_period("Q"), axis=1).mean().reset_index()

df_5 = df_4.drop(['RegionID', 'Metro', 'CountyName','SizeRank'], axis=1)

df_6 = df_5.set_index(['State', 'RegionName'])

df_6.columns =df_6.columns.map(lambda t: str(t).lower())

return df_6

In [15]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

In [96]:
univ_towns = get_list_of_university_towns()

In [17]:
housing = convert_housing_data_to_quarters()

In [18]:
housing = housing.reset_index()

In [97]:
univ_towns.head()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [98]:
housing['College Town'] = 'No'
univ_towns['College Town'] = 'Yes'

In [99]:
housing = housing.set_index('RegionName')
univ_towns = univ_towns.set_index('RegionName')

KeyError: 'RegionName'

In [100]:
housing.head(1)

,,State,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,...,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,College Town
State_Full,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,522800.0,528066.666667,532266.666667,540800.0,557200.0,572833.333333,582866.666667,591633.333333,587200.0,No


In [101]:
univ_towns.head(1)

,State,RegionName,College Town
1,Alabama,Auburn,Yes


In [24]:
#all_towns = pd.merge(housing, univ_towns, how='outer', left_index=True, right_index=True)

In [62]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}



In [76]:
def check_state(name):
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    if name in states:
        return states.get(name)

In [77]:
check_state('NH')

'New Hampshire'

In [78]:
housing['State_Full'] = housing['State'].apply(lambda x: check_state(x))

In [102]:

housing.head()

,,State,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,...,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,College Town
State_Full,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0,No
California,Los Angeles,CA,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,...,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0,No
Illinois,Chicago,IL,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,...,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0,No
Pennsylvania,Philadelphia,PA,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,...,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0,No
Arizona,Phoenix,AZ,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,...,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0,No


In [80]:
housing.reset_index(inplace=True)

In [88]:
housing.set_index(['State_Full', 'RegionName'], inplace = True)

In [106]:
univ_towns.head()

RegionName
State   College Town              
Alabama Yes                 Auburn
        Yes               Florence
        Yes           Jacksonville
        Yes             Livingston
        Yes             Montevallo

In [107]:
univ_towns.reset_index(inplace=True)

In [108]:

univ_towns.set_index(['State', 'RegionName'], inplace = True)

In [126]:
# pd.set_option('display.max_rows', 500)
# univ_towns

College Town
State         RegionName                   
Alabama       Auburn                    Yes
              Florence                  Yes
              Jacksonville              Yes
              Livingston                Yes
              Montevallo                Yes
              Troy                      Yes
              Tuscaloosa                Yes
              Tuskegee                  Yes
Alaska        Fairbanks                 Yes
Arizona       Flagstaff                 Yes
              Tempe                     Yes
              Tucson                    Yes
Arkansas      Arkadelphia               Yes
              Conway                    Yes
              Fayetteville              Yes
              Jonesboro                 Yes
              Magnolia                  Yes
              Monticello                Yes
              Russellville              Yes
              Searcy                    Yes
California    Angwin                    Yes
              Arcata                    Yes
              Berkeley                  Yes
              Chico                     Yes
              Claremont                 Yes
              Cotati                    Yes
              Davis                     Yes
              Irvine                    Yes
              Isla                      Yes
              Los                       Yes
              Merced                    Yes
              Orange                    Yes
              Palo                      Yes
              Pomona                    Yes
              Redlands                  Yes
              Riverside                 Yes
              Sacramento                Yes
              San                       Yes
              San                       Yes
              San                       Yes
              Santa                     Yes
              Santa                     Yes
              Turlock                   Yes
              Westwood,                 Yes
              Whittier                  Yes
Colorado      Alamosa                   Yes
              Boulder                   Yes
              Durango                   Yes
              Fort                      Yes
              Golden                    Yes
              Grand                     Yes
              Greeley                   Yes
              Gunnison                  Yes
              Pueblo                    Yes
Connecticut   Fairfield                 Yes
              Middletown                Yes
              New                       Yes
              New                       Yes
              New                       Yes
              Storrs                    Yes
              Willimantic               Yes
Delaware      Dover                     Yes
              Newark                    Yes
Florida       Ave                       Yes
              Boca                      Yes
              Coral                     Yes
              DeLand                    Yes
              Estero                    Yes
              Gainesville               Yes
              Orlando                   Yes
              Sarasota                  Yes
              St.                       Yes
              St.                       Yes
              Tallahassee               Yes
              Tampa                     Yes
Georgia       Albany                    Yes
              Athens                    Yes
              Atlanta                   Yes
              Carrollton                Yes
              Demorest                  Yes
              Fort                      Yes
              Kennesaw                  Yes
              Milledgeville             Yes
              Mount                     Yes
              Oxford                    Yes
              Rome                      Yes
              Savannah                  Yes
              Statesboro                Yes
              Valdosta                  Yes
              Waleska      

In [155]:
housing.head()

,State_Full,RegionName,State,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,...,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,College Town
0,New York,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0,No
1,California,Los Angeles,CA,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,...,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0,No
2,Illinois,Chicago,IL,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,...,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0,No
3,Pennsylvania,Philadelphia,PA,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,...,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0,No
4,Arizona,Phoenix,AZ,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,...,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0,No


In [145]:
housing.reset_index(inplace=True)
 
        

In [152]:
housing_location_only = housing[['State_Full', 'RegionName']]

In [153]:
housing_location_only.head()

,State_Full,RegionName
0,New York,New York
1,California,Los Angeles
2,Illinois,Chicago
3,Pennsylvania,Philadelphia
4,Arizona,Phoenix


In [170]:
def find_college_town(x,y):
    unique_states = univ_towns.index.get_level_values('State').unique().tolist()
    if x in unique_states:
        towns_in_state = univ_towns.loc[x].index.tolist()
        if y in towns_in_state:
            return 'Yes'
        else:
            return "No"

In [171]:
housing_location_only['College Town 2'] = housing_location_only.apply(lambda x: find_college_town(x['State_Full'], x['RegionName']), axis=1)

/Users/aschharwood/anaconda/envs/ipykernel_py2/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [176]:
housing_location_only

,State_Full,RegionName,College Town 2
0,New York,New York,None
1,California,Los Angeles,No
2,Illinois,Chicago,No
3,Pennsylvania,Philadelphia,No
4,Arizona,Phoenix,No
5,Nevada,Las Vegas,No
6,California,San Diego,No
7,Texas,Dallas,Yes
8,California,San Jose,No
9,Florida,Jacksonville,No


In [142]:
find_college_town('Texas')

['Abilene', 'Alpine', 'Austin', 'Beaumont', 'Canyon', 'College', 'Commerce', 'Dallas', 'Denton', 'Fort', 'Georgetown', 'Huntsville', 'Houston', 'Keene', 'Kingsville', 'Lubbock', 'Nacogdoches', 'Plainview', 'Prairie', 'San', 'Stephenville', 'Waco']


In [122]:
univ_towns.index.get_level_values('State').unique().tolist()

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'Wisconsin',
 'Wyoming']

In [ ]:
f.index.get_level_values('co').unique()

In [2]:
import pandas as pd

df = pd.read_csv('City_Zhvi_AllHomes.csv')

In [3]:
df.head(2)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100


In [6]:
regions = list(df.columns[1:3])

In [7]:
regions

['RegionName', 'State']